# 1. 先用預訓練好的神經網路提取影片特徵

In [3]:
import cv2
import os
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model

In [4]:
# VGG16 model，只做到特徵提取
base_model = VGG16(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

In [9]:
# 提取特徵函數
def extract_features(video_path, model):
    cap = cv2.VideoCapture(video_path)
    features = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (90, 160))  # resize
        frame = np.expand_dims(frame, axis=0)  # 拓展維度
        frame = preprocess_input(frame)  # 預處理
        feature = model.predict(frame)  # 提取特徵

        # TODO: 想一下要怎麼處理每一幀的feature，當作是這個影片的feature
        features.append(feature)
        # TODO: 改這裡

    cap.release()
    return np.concatenate(features, axis=0)

In [8]:
# 提取视频特征
dataset_dir = './dataset'

videos_features = [] # 存所有data特徵向量的特徵陣列
class_num = [] # 存每個類別的影片數量

for category in os.listdir(dataset_dir):
    video_count = 0 # 計算這個類別有幾個影片
    category_path = os.path.join(dataset_dir, category)
    for video in os.listdir(category_path):
        video_path = os.path.join(category_path, video)
        if video_path.is_dir():
            continue
        video_count += 1
        features = extract_features(video_path, model)
        videos_features.append(features)
    class_num.append(video_count)


1/1 [==============================] - 0s 46ms/step
(46, 10, 5, 512)


# 2. 再用DBSCAN分群

In [ ]:
from sklearn.cluster import DBSCAN

# video_features 是一個特徵陣列，每一行代表一個data的特徵向量
X = videos_features

# 調整DBSCAN參數
eps = 0.5  # 範圍
min_samples = 5  # 成為核心的最小點數

# 應用DBSCAN算法
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
labels = dbscan.fit_predict(X)

# labels 中的每个值表示对应数据点的簇标签，-1 表示噪声点

In [ ]:
# 分析分群結果
